In [1]:
!conda update scikit-learn --yes
!conda update sdv --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - scikit-learn


The following packages will be REMOVED:

  backports-1.0-pyhd8ed1ab_3
  backports.functools_lru_cache-1.6.5-pyhd8ed1ab_0
  jmespath-1.0.1-pyhd8ed1ab_0
  libgomp-13.2.0-h807b86a_2

The following packages will be UPDATED:

  libgcc-ng                               13.2.0-h807b86a_2 --> 13.2.0-h807b86a_3 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

PackageNotInstalledError: Package is not installed in prefix.
  prefix: /home/ec2-user/anaconda3/envs/python3
  package name: sdv




In [3]:
import os
import sys

# sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
# os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETs_Experiment

In [4]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [5]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt  education  educational-num marital-status  \
0   33       NaN  119918  Bachelors               13  Never-married   

  occupation   relationship   race gender  capital-gain  capital-loss  \
0        NaN  Not-in-family  Black   Male             0             0   

   hours-per-week native-country income  
0              45            NaN  <=50K  


In [6]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Missingist (Drop): Dropped 2872 rows.
Preprocessor - Outlierist (IQR): Dropped  3023 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   223 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1744 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1055 rows on fnlwgt         . Kept [-63698.0, 419566.0] only.
Preprocessor - Outlierist (IQR): Dropped   241 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6086 in 36201 rows.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column occupation      bee

In [7]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

ModuleNotFoundError: No module named 'sdv'

In [6]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding income.
   age  workclass    fnlwgt education  educational-num      marital-status  \
0   58  Local-gov  226802.0   HS-grad               11  Married-civ-spouse   

         occupation relationship   race gender  capital-gain  capital-loss  \
0  Transport-moving    Unmarried  White   Male             0             0   

   hours-per-week native-country income  
0              30     Yugoslavia  <=50K  


In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
import PETs_Experiment

para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

====== ====== ====== ====== ====== ======
Executor: Total execution time: 18.9125 sec.
====== ====== ====== ====== ====== ======
[Adt Income] adult.csv
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'   : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'      : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 2
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 1
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()



# for k ,v in executor.loader.items():
#     print(k)
#     print(v.data.head(1))
    

KeyboardInterrupt: 

In [ ]:
executor.

In [4]:
test = [6 ,4]
trial = 1

test[:(trial-1)]

[]

In [28]:




from PETs_Experiment import Loader
self = executor    


def test(key ,para):
    print(key)
    return key , 'para'




with ThreadPoolExecutor() as async_executor:
    futures = [async_executor.submit(test, _key, _para)
                for _key ,_para in self.para['Loader_setting'].items()]

    for future in futures:
        key, loader_result = future.result()
        self.loader[key] = loader_result

self.loader

adult
NHANES


{'adult': 'para', 'NHANES': 'para'}